In [1]:
import os
import re
import time
from datetime import datetime

import requests

from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [15]:
import pprint

In [2]:
page_url = 'https://www.bexar.org/722/Jail-Activity-Reports'
xpath = '//a[text()="State and County"]'

In [6]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.get(page_url)
time.sleep(1)
link_elements = driver.find_elements_by_xpath(xpath)
links = [element.get_attribute('href') for element in link_elements]
driver.quit()

In [7]:
print(links)

['http://apps.bexar.org/jailactivity/KB50061D20191017.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191018.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191019.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191020.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191021.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191022.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191023.txt']


In [33]:
links = links[0:1]
for link in links:
    print(link)
    output_folder = os.path.expanduser('~/data/jailactivityreports')
    output_file_path = f'{output_folder}/{os.path.basename(link)}'
    report = requests.get(link).text
    line_start_pattern = re.compile('^0[A-Z]\d{9}')
    w_pattern = re.compile('W\d{7}')
    j_pattern = re.compile('J\d{7}')
    whitespace_pattern = re.compile('^\s{68}')
    date_pattern = re.compile('\d{2}/\d{2}/\d{4} \d{4}\s{43}')
    lines = report.split('\r\n')
#     print(lines[:20])
    for i, line in enumerate(lines, 1):
        if line_start_pattern.match(line):
            print(f'{i}>>>{line}')
            record = {}
            charges = []
            record['booking_number'] = line[0:11]
            booking_date = datetime.strptime(line[14:29], '%m %d %Y %H%M')
            name = line[30:63].rstrip()
            print(booking_number, booking_date, name)
        
        elif w_pattern.search(line) or j_pattern.search(line):
            try:
                action_date = datetime.strptime(line[10:25], '%m/%d/%Y %H%M')
            except:
                pass
            w_number = line[26:34].rstrip()
            j_number = line[35:43].rstrip()
            court= line[44:48]
            case_number = line[49:60].rstrip()
            charge = line[68:91].rstrip()
            action = line[92:110].rstrip()
            action_agent = line[110:122].rstrip()
            charges.append([charge, action, action_agent])
            print(action_date, w_number, j_number, court, case_number, charges)
        
        elif whitespace_pattern.match(line):
            print(">>>? Got to whitespace checking line")
            charge = line[68:91].rstrip()
            action = line[92:110].rstrip()
            action_agent = line[110:122].rstrip()
            charges.append([charge, action, action_agent])
            print(action_date, w_number, j_number, court, case_number, charges)
            
        elif date_pattern.search(line):
            action_date = datetime.strptime(line[10:25], '%m/%d/%Y %H%M')
            charge = line[68:91].rstrip()
            action = line[92:110].rstrip()
            action_agent = line[110:122].rstrip()
            charges.append([charge, action, action_agent])
            print(action_date, w_number, j_number, court, case_number, charges)
#         try:
#             print(booking_number, booking_date, name, action_date, w_number, j_number, court, case_number, charge, action)
#         except:
#             pass
        
#             print(f'{i}>>>{line}')
#     with open(output_file_path, 'w') as output_file:
#         output_file.write(report)

http://apps.bexar.org/jailactivity/KB50061D20191017.txt
6>>>0B201847280   10 02 2018 0120 WELCH, RICHARD HERMAN            W M 03 29 81 6311 BIG VALLEY        SAN ANTONIO, TX. 78242
0B201921067 2018-10-02 01:20:00 WELCH, RICHARD HERMAN
2019-10-16 03:22:00  J1871706 NM   407384 [['F2 ONLINE SOL MINOR UND', 'CASE DISMISSED', '']]
2019-10-16 03:22:00  J1871707 D186 2018CR13481 [['F2 ONLINE SOL MINOR UND', 'CASE DISMISSED', ''], ['F1 CONTINUOUS SEX ABUSE', 'CASE DISMISSED', 'JD ANGELINI']]
10>>>0B201858574   12 10 2018 1008 SURRATT, DARYL                   B M 07 13 69  HAVEN FOR HOPE        SAN ANTONIO, TX. 78220   TX0150000
0B201921067 2018-12-10 10:08:00 SURRATT, DARYL
2019-10-16 04:20:00 W1671671 J1712077 D379 2016CR7440 [['F3 STALKING', 'TRANSFER TO', 'MIOF']]
13>>>0B201910525   03 05 2019 0323 BARRY, JOSEPH ANTHONY            W M 06 28 91 215 GROSVENOR          SAN ANTONIO, TX. 78221   TX0150000
0B201921067 2019-03-05 03:23:00 BARRY, JOSEPH ANTHONY
2019-10-16 02:59:00  J1896220 D399 

2019-10-16 00:38:00  J1935200 NM   540723 [['F1 POSS W/I DEL CS PG1', "REL'D ON BOND", 'CPL EDENS']]
2019-10-16 00:38:00  J1935201 NM   540725 [['F1 POSS W/I DEL CS PG1', "REL'D ON BOND", 'CPL EDENS'], ['F1 POSS W/I DEL CS PG1', "REL'D ON BOND", 'CPL EDENS']]
451>>>0B201946376   10 15 2019 0559 WILSON, VINCENT CORNELL          B M 06 15 85 9518 HEIDELBERG        SAN ANTONIO, TX. 78233   TX0150000
0B201921067 2019-10-15 05:59:00 WILSON, VINCENT CORNELL
2019-10-16 03:03:00  J1935203 NM   540728 [['F3 DRIVING WHILE INTOXI', "REL'D ON BOND", 'CANAIL 4399']]
454>>>0B201946434   10 15 2019 1355 HOUTS, KYLE MATTHEW              W M 02 11 85 1131 HOU TOWN          EDINBURG, TX. 78541      TX0150000
0B201921067 2019-10-15 13:55:00 HOUTS, KYLE MATTHEW
2019-10-16 21:46:00  J1935282 CC15 625388 [['MA DWI W/BAC 0.15 OR HI', "REL'D ON BOND", 'MENDOZA']]
457>>>0B201946449   10 15 2019 1802 RODRIGUEZ, OSCAR                 W M 08 14 00 642 37TH               SAN ANTONIO, TX. 78237   TX0150000
0B201921

0B201921067 2019-10-16 14:12:00 PANTOJA, ROBERTO MUNOZ
2019-10-16 20:33:00 W1707890 J1923034 D187 2019CR11377 [['F2 SEXUAL ASSAULT', "REL'D ON BOND", 'PILIADO']]
868>>>0B201946588   10 16 2019 1414 OZUNA, SAVANNAH ANN              L F 01 14 94 5806 CASTLE BROOK      SAN ANTONIO, TX. 78218   TX0150000
0B201921067 2019-10-16 14:14:00 OZUNA, SAVANNAH ANN
2019-10-16 20:33:00 W1701500 J1873909 CC9  598301 [['MB POSS MARIHUANA 0-2 O', 'AWTG DKT APPEARNCE', '']]
2019-10-16 20:33:00 W1703275 J1930457 CM   063219 [['MB POSS MARIHUANA 0-2 O', 'AWTG DKT APPEARNCE', ''], ['F3 POSS CS PG 1  1 GRAM', 'AWTG FILING', '']]
872>>>0B201946589   10 16 2019 1457 TREVINO, JENNY JAZIAL            L F 11 10 84 934 BRAD               SAN ANTONIO, TX. 78201   TX0150000
0B201921067 2019-10-16 14:57:00 TREVINO, JENNY JAZIAL
2019-10-16 14:57:00 W1699721 J1882306 CC15 620210 [['MB FAIL TO STOP GIVE IN', "REL'D ON BOND", 'MCLAURIN']]
875>>>0B201946590   10 16 2019 1741 MOORE, YVETTE MICHELLE           B F 04 28 67 7